In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot_2samples
import seaborn as sns
import geopy.distance


df = pd.read_csv("cougars_and_coyotes.csv")
df.info()

/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579604 entries, 0 to 579603
Data columns (total 18 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   event-id                         579604 non-null  int64  
 1   visible                          579604 non-null  bool   
 2   timestamp                        579604 non-null  object 
 3   location-long                    198705 non-null  float64
 4   location-lat                     198705 non-null  float64
 5   acceleration-raw-x               0 non-null       float64
 6   acceleration-raw-y               0 non-null       float64
 7   acceleration-x                   380899 non-null  float64
 8   acceleration-y                   380899 non-null  float64
 9   external-temperature             1361 non-null    float64
 10  gps:fix-type                     3778 non-null    float64
 11  gps:satellite-count              1361 non-null    float64
 12  ta

In [2]:
df.drop(['event-id','visible','acceleration-raw-x','acceleration-raw-y','acceleration-x','acceleration-y','external-temperature','gps:fix-type','gps:satellite-count','tag-tech-spec','sensor-type','individual-taxon-canonical-name','study-name'],axis=1,inplace=True)
df = df.dropna(axis = 0, subset = ['location-long'])
df = df.dropna(axis = 0, subset = ['location-lat'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198705 entries, 0 to 198704
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   timestamp                    198705 non-null  object 
 1   location-long                198705 non-null  float64
 2   location-lat                 198705 non-null  float64
 3   tag-local-identifier         198705 non-null  object 
 4   individual-local-identifier  198705 non-null  object 
dtypes: float64(2), object(3)
memory usage: 9.1+ MB


In [4]:
identifiers = df['individual-local-identifier'].unique()
column_names = ['Carnivore ID','Date','Step length','Turning angle','dx','dy']
# new_df = pd.DataFrame(columns = column_names) 

def angle_between(p1, p2):
    ang1 = np.arctan2(p1[0],p1[1])
    ang2 = np.arctan2(p2[0],p2[1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

i=0
new_data=[]
for identifier in identifiers:
    i+=1
    name_updated = 'c' + str(i) 
    search = df.loc[df['individual-local-identifier']==identifier]
    prev = None
    for index, row in search.iterrows():
        if prev is not None:
            dx = geopy.distance.distance((prev['location-lat'],prev['location-long']), (row['location-lat'],prev['location-long'])).km
            if prev['location-lat'] > row['location-lat']:
                dx = dx*-1
            dy = geopy.distance.distance((prev['location-lat'],prev['location-long']), (prev['location-lat'],row['location-long'])).km
            if prev['location-long'] > row['location-long']:
                dy = dy*-1
            turning_angle = angle_between((prev['location-lat'],prev['location-long']),(row['location-lat'],row['location-long'])) #p2 curr
            result = (name_updated,row['timestamp'],(((dx**2)+(dy**2))**0.5),turning_angle,dx,dy)
            new_data.append(result)
        prev = row
        
new_df = pd.DataFrame(new_data, columns =column_names)        

In [5]:
new_df.to_csv('carnivores.csv', index=False)